In [16]:
import pandas as pd

train = pd.read_csv('train.csv')
print('Train shape:', train.shape)
print(train.head())

Train shape: (913000, 4)
         date  store  item  sales
0  2013-01-01      1     1     13
1  2013-01-02      1     1     11
2  2013-01-03      1     1     14
3  2013-01-04      1     1     13
4  2013-01-05      1     1     10


In [17]:
test = pd.read_csv('test.csv')
print('Train columns:', train.columns.tolist())
print('Test columns:', test.columns.tolist())

sample_submission = pd.read_csv('sample_submission.csv')
print(sample_submission.head())

Train columns: ['date', 'store', 'item', 'sales']
Test columns: ['id', 'date', 'store', 'item']
   id  sales
0   0     52
1   1     52
2   2     52
3   3     52
4   4     52


In [18]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor()
rf.fit(X=train[['store', 'item']], y=train['sales'])

RandomForestRegressor()

In [19]:
test['sales'] = rf.predict(test[['store', 'item']])
test[['id', 'sales']].to_csv('submission.csv', index=False)

In [20]:
# mini conda install xgboost on mac m1
# !conda install -c conda-forge xgboost --yes

In [21]:
import xgboost as xgb

dtrain = xgb.DMatrix(data=train[['store', 'item']], label=train['sales'])

params = {'objective':'reg:linear', 'max_depth':2, 'verbosity': 0}
xg_depth_2 = xgb.train(params=params, dtrain=dtrain)

params = {'objective':'reg:linear', 'max_depth':8, 'verbosity': 0}
xg_depth_8 = xgb.train(params=params, dtrain=dtrain)

params = {'objective':'reg:linear', 'max_depth':15, 'verbosity': 0}
xg_depth_15 = xgb.train(params=params, dtrain=dtrain)

In [22]:
from sklearn.metrics import mean_squared_error

dtrain = xgb.DMatrix(data=train[['store', 'item']])
dtest = xgb.DMatrix(data=test[['store', 'item']])

# For each of 3 trained models
for model in [xg_depth_2, xg_depth_8, xg_depth_15]:
    # Make predictions
    train_pred = model.predict(dtrain)     
    test_pred = model.predict(dtest)          
    
    # Calculate metrics
    mse_train = mean_squared_error(train['sales'], train_pred)                  
    mse_test = mean_squared_error(test['sales'], test_pred)
    print('MSE Train: {:.3f}. MSE Test: {:.3f}'.format(mse_train, mse_test))

MSE Train: 607.487. MSE Test: 355.796
MSE Train: 294.466. MSE Test: 42.799
MSE Train: 255.334. MSE Test: 3.666
